<a href="https://colab.research.google.com/github/Shrinjita/Bajaj-Finserv-Q-/blob/main/bfh_q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import cv2
import pytesseract
import os
import numpy as np
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, threshold_img = cv2.threshold(gray_img, 150, 255, cv2.THRESH_BINARY)
    denoised_img = cv2.GaussianBlur(threshold_img, (5, 5), 0)
    dpi_res = (int(img.shape[1] * 300 / 72), int(img.shape[0] * 300 / 72))
    resized_img = cv2.resize(denoised_img, dpi_res)
    return resized_img
def extract_text_with_ocr(image_path):
    preprocessed_image = preprocess_image(image_path)
    extracted_text = pytesseract.image_to_string(preprocessed_image)
    return extracted_text
def process_images_in_folder(folder_path, output_file):
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    with open(output_file, 'w') as file:
        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)
            try:
                extracted_text = extract_text_with_ocr(image_path)
                file.write(f"Results for {image_file}:\n")
                file.write(extracted_text)
                file.write("\n" + "=" * 50 + "\n\n")
            except Exception as e:
                file.write(f"Error processing {image_file}: {e}\n")
                file.write("\n" + "=" * 50 + "\n\n")
def main():
    folder_path = '/content/drive/MyDrive/Colab Notebooks/demo'
    output_file = '/content/drive/MyDrive/Colab Notebooks/demo2/output.txt'
    process_images_in_folder(folder_path, output_file)
if __name__ == '__main__':
    main()

In [38]:
input = '/content/drive/MyDrive/Colab Notebooks/demo2/output.txt'

In [39]:
with open(input, 'r', encoding='utf-8') as file:
  text = file.read()

In [46]:
keywords = 'Ref. Doctor'

In [51]:
output_json_file = '/content/drive/MyDrive/Colab Notebooks/demo2/output.json'

In [59]:
import json

In [69]:
if keywords in text:
  chunks = text.split("\n")
  chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
  for chunk in chunks:
    if keywords in chunk:
      print(chunk)

Ref. Doctor : Or. RUPA BANERSEE
Ref. Doctor : Or. RUPA BANERJEE
Ref. Doctor ¢ Dr. Siddhant Mathu Mobile No pT
Ref. Doctor : Dr. Siddhant Matour Mobile No pe
Ref. Doctor : Dr. Siddhant Mathur Mobile No po


In [79]:
import json

input_file = '/content/drive/MyDrive/Colab Notebooks/demo2/output.txt'
output_json_file = '/content/drive/MyDrive/Colab Notebooks/demo2/output.json'

try:
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()

    keywords = [
        "PROTHROMBIN TIME", "INR", "APTT", "URIC ACID", "SERUM CREATININE",
        "RANDOM BLOOD GLUCOSE", "RBS", "BILIRUBIN TOTAL", "CONJUGATED", "UNCONJUGATED",
        "SGOT", "SGPT", "ALKALINE PHOSPHATASE", "TOTAL PROTEIN", "ALBUMIN",
        "GLOBULIN", "A/G RATIO", "GAMMAT-GT"
    ]
    data = {}
    chunks = text.split("\n")
    chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
    for keyword in keywords:
      if keywords in chunk:
        data[f"test_name"] = chunk

        with open(output_json_file, 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"JSON data written to {output_json_file}")
    else:
        print(f"Keyword '{keywords}' not found in the input text file.")

except FileNotFoundError:
    print(f"Error: Input file '{input_file}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: 'in <string>' requires string as left operand, not list


In [76]:
import re
import json
from typing import List, Dict, Any

def extract_test_results(text: str) -> List[Dict[str, Any]]:
    results = []
    test_names = [
        "PROTHROMBIN TIME", "INR", "APTT", "URIC ACID", "SERUM CREATININE",
        "RANDOM BLOOD GLUCOSE", "RBS", "BILIRUBIN TOTAL", "CONJUGATED", "UNCONJUGATED",
        "SGOT", "SGPT", "ALKALINE PHOSPHATASE", "TOTAL PROTEIN", "ALBUMIN",
        "GLOBULIN", "A/G RATIO", "GAMMAT-GT"
    ]
    lines = text.split('\n')

    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        for test_name in test_names:
            if test_name in line:

                patterns = [

                    rf'{test_name}\s+([\d\.]+)\s*([HL]?)\s*([a-zA-Z/]+)\s+([\d\.\s\-<>]+)',

                    rf'([A-Z][A-Z\s\(\)\-\.\/]*{test_name}[A-Z\s\(\)\-\.\/]*)\s+([\d\.]+)\s*([HL]?)\s*([a-zA-Z/]+)\s+([\d\.\s\-<>]+)',


                    rf'{test_name}\s+([\d\.]+)\s*([HL]?)\s*(?:Normal)?\s*(<[\d\.]+)',
                ]

                match = None
                for pattern in patterns:
                    match = re.search(pattern, line)
                    if match:
                        break

                if match:
                    process_test_match(match, test_name, results)


    direct_pattern = r'([A-Z][A-Z\s\(\)\-\.\/]*)\s+([\d\.]+)\s*([HL]?)\s*([a-zA-Z/]*)\s+([\d\.\s\-<>]+)'
    for line in lines:
        if any(keyword in line for keyword in ["BILIRUBIN", "SGOT", "SGPT", "CREATININE", "GLUCOSE", "PROTEIN", "ALBUMIN", "INR"]):
            match = re.search(direct_pattern, line)
            if match:
                extracted_name = match.group(1).strip()
                process_test_match(match, extracted_name, results)


    special_case_extraction(text, results)


    return standardize_results(results)

In [75]:
def process_test_match(match, test_name: str, results: List[Dict[str, Any]]) -> None:
    """Process a regex match for a test and add to results"""
    try:
        if len(match.groups()) >= 2:
            if test_name in match.group(0):

                extracted_name = test_name
                value_str = match.group(1)
                flag = match.group(2) if len(match.groups()) > 1 else ""
                unit = match.group(3) if len(match.groups()) > 2 else ""
                range_str = match.group(4) if len(match.groups()) > 3 else ""
            else:

                extracted_name = match.group(1).strip()
                value_str = match.group(2)
                flag = match.group(3) if len(match.groups()) > 2 else ""
                unit = match.group(4) if len(match.groups()) > 3 else ""
                range_str = match.group(5) if len(match.groups()) > 4 else ""


            test_value = float(value_str.strip())


            out_of_range = flag.strip() in ['H', 'L']


            bio_reference_range = process_reference_range(range_str.strip())

            results.append({
                "test_name": extracted_name,
                "test_value": test_value,
                "test_unit": unit.strip(),
                "lab_test_out_of_range": out_of_range,
                "bio_reference_range": bio_reference_range
            })
    except Exception:

        pass

def process_reference_range(range_str: str) -> str:
    """Process and standardize reference range string"""

    range_match = re.search(r'([\d\.]+)\s*-\s*([\d\.]+)', range_str)
    if range_match:
        return f"{range_match.group(1)} - {range_match.group(2)}"


    less_than_match = re.search(r'<\s*([\d\.]+)', range_str)
    if less_than_match:
        return f"0 - {less_than_match.group(1)}"

    return range_str

def special_case_extraction(text: str, results: List[Dict[str, Any]]) -> None:
    """Extract results using specialized patterns for specific tests"""

    inr_pattern = r'INR\s+([\d\.]+)\s*([HL]?)\s*Normal\s*<([\d\.]+)'
    inr_match = re.search(inr_pattern, text)
    if inr_match and not any(r["test_name"] == "INR" for r in results):
        results.append({
            "test_name": "INR",
            "test_value": float(inr_match.group(1)),
            "test_unit": "",
            "lab_test_out_of_range": inr_match.group(2) in ['H', 'L'],
            "bio_reference_range": f"0 - {inr_match.group(3)}"
        })


    bilirubin_patterns = [
        r'BILIRUBIN TOTAL\s+([\d\.]+)\s*([HL]?)\s*([a-zA-Z/]+)\s+([\d\.\s\-]+)',
        r'CONJUGATED[^:]*\s+([\d\.]+)\s*([HL]?)\s*([a-zA-Z/]+)\s+([\d\.\s\-]+)',
        r'UNCONJUGATED[^:]*\s+([\d\.]+)\s*([HL]?)\s*([a-zA-Z/]+)\s+([\d\.\s\-]+)'
    ]

    for pattern in bilirubin_patterns:
        match = re.search(pattern, text)
        if match:
            test_name = "BILIRUBIN TOTAL" if "TOTAL" in pattern else "CONJUGATED" if "CONJUGATED" in pattern else "UNCONJUGATED"
            if not any(r["test_name"] == test_name for r in results):
                results.append({
                    "test_name": test_name,
                    "test_value": float(match.group(1)),
                    "test_unit": match.group(3),
                    "lab_test_out_of_range": match.group(2) in ['H', 'L'],
                    "bio_reference_range": process_reference_range(match.group(4))
                })


    glucose_pattern = r'(?:RANDOM BLOOD GLUCOSE|BLOOD GLUCOSE|RBS)\s+([\d\.]+)\s*([HL]?)\s*([a-zA-Z/]+)\s+([\d\.\s\-]+)'
    glucose_match = re.search(glucose_pattern, text)
    if glucose_match and not any(r["test_name"] == "RANDOM BLOOD GLUCOSE" for r in results):
        results.append({
            "test_name": "RANDOM BLOOD GLUCOSE",
            "test_value": float(glucose_match.group(1)),
            "test_unit": glucose_match.group(3),
            "lab_test_out_of_range": glucose_match.group(2) in ['H', 'L'],
            "bio_reference_range": process_reference_range(glucose_match.group(4))
        })